In [1]:
import importlib
import gym_model
importlib.reload(gym_model)
from visualisation import JupyterVizGym, GymDrawer
from optimization import LayoutTemplate
import random

In [10]:
model_params = {
    "interarrival_time": {
        "type": "SliderInt",
        "value": 5,
        "label": "time between arrivals:",
        "min": 1,
        "max": 50,
        "step": 1,
    },
    "agent_exercise_duration": {
        "type": "SliderInt",
        "value": 30,
        "label": "mean exercise duration:",
        "min": 1,
        "max": 3 * 60,
        "step": 1,
    },
}

RANDOM_LAYOUT = False

if RANDOM_LAYOUT:
    model_params["machine_density"] ={
        "type": "SliderFloat",
        "value": 0.25,
        "label": "Density of gym equipment:",
        "min": 0.05,
        "max": 1,
        "step": 0.05,
    }
else:
    template = LayoutTemplate.square_isles(isle_rows=2, isle_cols=3)
    model_params["layout"] = template.instantiate([random.choice(list(gym_model.Equipment)) for _ in range(len(template))])
    model_params["spawn_location"] = template.entrance


page = JupyterVizGym(
    gym_model.Gym,
    model_params,
    measures=["Utilization"],
    name="Test gym",
    agent_portrayal=lambda agent: agent.portrayal,
    # space_drawer="default",
    space_drawer=GymDrawer,
)

page

Cannot show ipywidgets in text

In [11]:
gym = gym_model.Gym(interarrival_time=10, agent_exercise_duration=30)

model_df, agent_df = gym.run(1000, progress_bar=True)
agent_df

Simulating gym: 100%|██████████| 1000/1000 [00:00<00:00, 1443.09it/s, 15 agents]


State
Step AgentID             
0    0          SEARCHING
1    0        WORKING_OUT
2    0        WORKING_OUT
3    0        WORKING_OUT
4    0        WORKING_OUT
...                   ...
999  96       WORKING_OUT
     97       WORKING_OUT
     98       WORKING_OUT
     99       WORKING_OUT
     100        SEARCHING

[13335 rows x 1 columns]